# Working with Continuous Data

The specific dataframe methods we will use that we haven't covered before are:

* `replace()`
* `value_counts()`
* `crosstab()`


In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# For slightly nicer charts
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 150

***
For this section we will use a selection from the dataset `fandango_score_comparison.csv` ([source](https://github.com/fivethirtyeight/data/tree/master/fandango)), containing data about movie ratings from Fandango, Rotten Tomatoes, IMDB, and Metacritic. The dataset was used for this brief article, [Be Suspicious Of Online Movie Ratings, Especially Fandango’s](https://fivethirtyeight.com/features/fandango-movies-ratings/), published by [fivethirtyeight](https://fivethirtyeight.com/).

First, we read a CSV file of the movie ratings and assign it to a variable called `df_full`. We will then select a subset of the data and assign the subsection to the variable `df`.
Each row in the dataset represents one movie.  The columns are labelled as follows: 

* FILM: The film in question
* RottenTomatoes: The Rotten Tomatoes Tomatometer score for the film
* RottenTomatoes_User: The Rotten Tomatoes user score for the film
* Metacritic: The Metacritic critic score for the film
* Metacritic_User: The Metacritic user score for the film
* IMDB: The IMDb user score for the film
* Fandango_Stars: The number of stars the film had on its Fandango movie page
* Fandango_Ratingvalue: The Fandango ratingValue for the film, as pulled from the HTML of each page. This is the actual average score the movie obtained.



In [3]:
df_full = pd.read_csv("fandango_score_comparison.csv")
df = df_full.loc[:,'FILM':'Fandango_Ratingvalue'] # this creates a subset of the dataframe using location baed indexing, which we will discuss later 
df.head()

,FILM,RottenTomatoes,RottenTomatoes_User,Metacritic,Metacritic_User,IMDB,Fandango_Stars,Fandango_Ratingvalue
0,Avengers: Age of Ultron (2015),74,86,66,7.1,7.8,5.0,4.5
1,Cinderella (2015),85,80,67,7.5,7.1,5.0,4.5
2,Ant-Man (2015),80,90,64,8.1,7.8,5.0,4.5
3,Do You Believe? (2015),18,84,22,4.7,5.4,5.0,4.5
4,Hot Tub Time Machine 2 (2015),14,28,29,3.4,5.1,3.5,3.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 8 columns):
FILM                    146 non-null object
RottenTomatoes          146 non-null int64
RottenTomatoes_User     146 non-null int64
Metacritic              146 non-null int64
Metacritic_User         146 non-null float64
IMDB                    146 non-null float64
Fandango_Stars          146 non-null float64
Fandango_Ratingvalue    146 non-null float64
dtypes: float64(4), int64(3), object(1)
memory usage: 9.2+ KB


Our dataset has 146 movies (one per row) and eight columns. The first column labelled `'FILM'` contains strings with the name of the Film and its year of release. The remaining columns are either integers or floats of representing various kinds of ratings of the quality of the film.

***

Before we start looking at how the ratings are related to one another, let's look at some descriptive statistics.

In [5]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
RottenTomatoes,146.0,60.849315,30.168799,5.0,31.25,63.50,89.0,100.0
RottenTomatoes_User,146.0,63.876712,20.024430,20.0,50.00,66.50,81.0,94.0
Metacritic,146.0,58.808219,19.517389,13.0,43.50,59.00,75.0,94.0
Metacritic_User,146.0,6.519178,1.510712,2.4,5.70,6.85,7.5,9.6
IMDB,146.0,6.736986,0.958736,4.0,6.30,6.90,7.4,8.6
Fandango_Stars,146.0,4.089041,0.540386,3.0,3.50,4.00,4.5,5.0
Fandango_Ratingvalue,146.0,3.845205,0.502831,2.7,3.50,3.90,4.2,4.8


Notice, the variations in the mean, min, and max values. Some of the ratings systems appear to be using 100 points scales, others 10 points scales, and others 5 points scales. Also, no matter what the scale, the average rating tends to be just a bit higher than the mid-point of the scale. 

The different scales make looking at the descriptive statistics a bit confusing so let's convert all the scales to be the same. This conversion process is commonly called 'normalizing' your data. In this case we are going to convert everything to a 10-point scale. 

We are NOT going to 'copy over' the existing values. Instead we are going to calculate new values and assign them to new columns.  

First, we will normalize the 100-point columns and assign them to new columns. 

In [6]:
hundred_list = ['RottenTomatoes', 'RottenTomatoes_User', 'Metacritic']
df[['RottenTomatoes_Norm', 'RottenTomatoes_User_Norm', 'Metacritic_Norm']] = df[hundred_list]/10
df.head()

,FILM,RottenTomatoes,RottenTomatoes_User,Metacritic,Metacritic_User,IMDB,Fandango_Stars,Fandango_Ratingvalue,RottenTomatoes_Norm,RottenTomatoes_User_Norm,Metacritic_Norm
0,Avengers: Age of Ultron (2015),74,86,66,7.1,7.8,5.0,4.5,7.4,8.6,6.6
1,Cinderella (2015),85,80,67,7.5,7.1,5.0,4.5,8.5,8.0,6.7
2,Ant-Man (2015),80,90,64,8.1,7.8,5.0,4.5,8.0,9.0,6.4
3,Do You Believe? (2015),18,84,22,4.7,5.4,5.0,4.5,1.8,8.4,2.2
4,Hot Tub Time Machine 2 (2015),14,28,29,3.4,5.1,3.5,3.0,1.4,2.8,2.9


Second, we will normalize the 5-point columns and assign them to new columns. 

In [7]:
five_list = ['Fandango_Stars', 'Fandango_Ratingvalue']
df[['Fandango_Stars_Norm', 'Fandango_Ratingvalue_Norm']] = df[five_list]*2
df.head()

,FILM,RottenTomatoes,RottenTomatoes_User,Metacritic,Metacritic_User,IMDB,Fandango_Stars,Fandango_Ratingvalue,RottenTomatoes_Norm,RottenTomatoes_User_Norm,Metacritic_Norm,Fandango_Stars_Norm,Fandango_Ratingvalue_Norm
0,Avengers: Age of Ultron (2015),74,86,66,7.1,7.8,5.0,4.5,7.4,8.6,6.6,10.0,9.0
1,Cinderella (2015),85,80,67,7.5,7.1,5.0,4.5,8.5,8.0,6.7,10.0,9.0
2,Ant-Man (2015),80,90,64,8.1,7.8,5.0,4.5,8.0,9.0,6.4,10.0,9.0
3,Do You Believe? (2015),18,84,22,4.7,5.4,5.0,4.5,1.8,8.4,2.2,10.0,9.0
4,Hot Tub Time Machine 2 (2015),14,28,29,3.4,5.1,3.5,3.0,1.4,2.8,2.9,7.0,6.0


Now we are going to drop all of the non-normed columns.

In [8]:
non_normed_ratings_list = ['RottenTomatoes', 'RottenTomatoes_User', 'Metacritic', 'Fandango_Stars', 'Fandango_Ratingvalue']
df = df.drop(columns=non_normed_ratings_list)
df.head()

,FILM,Metacritic_User,IMDB,RottenTomatoes_Norm,RottenTomatoes_User_Norm,Metacritic_Norm,Fandango_Stars_Norm,Fandango_Ratingvalue_Norm
0,Avengers: Age of Ultron (2015),7.1,7.8,7.4,8.6,6.6,10.0,9.0
1,Cinderella (2015),7.5,7.1,8.5,8.0,6.7,10.0,9.0
2,Ant-Man (2015),8.1,7.8,8.0,9.0,6.4,10.0,9.0
3,Do You Believe? (2015),4.7,5.4,1.8,8.4,2.2,10.0,9.0
4,Hot Tub Time Machine 2 (2015),3.4,5.1,1.4,2.8,2.9,7.0,6.0


So that looks great but our column order is bit jumbled. Let's do a quick fix to get the ratings in alphabetical order. We are going to need a list with the labels in the proper order first. 

In [15]:
# we can make a list manually
label_order_list_manual = ['FILM', 'Fandango_Ratingvalue_Norm', 'Fandango_Stars_Norm', 'IMDB', 'Metacritic_Norm', 'Metacritic_User', 'RottenTomatoes_Norm', 'RottenTomatoes_User_Norm']

# or we can make a list using a few properties and methods
label_order_list_fancy = df.columns.tolist()
label_order_list_fancy.sort() #Note that the sort() method sorts the list 'in place' and does not return anything, therefore it cannot be assigned to a variable

print('Manual result:', label_order_list_manual)
print('Fancy  result:', label_order_list_fancy)

Manual result: ['FILM', 'Fandango_Ratingvalue_Norm', 'Fandango_Stars_Norm', 'IMDB', 'Metacritic_Norm', 'Metacritic_User', 'RottenTomatoes_Norm', 'RottenTomatoes_User_Norm']
Fancy  result: ['FILM', 'Fandango_Ratingvalue_Norm', 'Fandango_Stars_Norm', 'IMDB', 'Metacritic_Norm', 'Metacritic_User', 'RottenTomatoes_Norm', 'RottenTomatoes_User_Norm']


For small datasets, manually creating lists may make sense and may be an efficient use of your time. However, it will often be the case where writing code to automate these types of processes will save you time and make your process less error prone. This will be particularly true for large datasets. Another advatage of solving your problems with code is the code can often bre reusable. The code for the 'fancy' approach above could be used with any dataset, while the manually created list will likely only ever be useful this one specific time. 

So we now have our list (we can use either), let's use it to 'reorder' our dataframe.

In [19]:
df = df[label_order_list_fancy]
df.head()

,FILM,Fandango_Ratingvalue_Norm,Fandango_Stars_Norm,IMDB,Metacritic_Norm,Metacritic_User,RottenTomatoes_Norm,RottenTomatoes_User_Norm
0,Avengers: Age of Ultron (2015),9.0,10.0,7.8,6.6,7.1,7.4,8.6
1,Cinderella (2015),9.0,10.0,7.1,6.7,7.5,8.5,8.0
2,Ant-Man (2015),9.0,10.0,7.8,6.4,8.1,8.0,9.0
3,Do You Believe? (2015),9.0,10.0,5.4,2.2,4.7,1.8,8.4
4,Hot Tub Time Machine 2 (2015),6.0,7.0,5.1,2.9,3.4,1.4,2.8


That looks pretty good. Let's rerun our descriptives and take another look at the variables.

In [20]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Fandango_Ratingvalue_Norm,146.0,7.690411,1.005662,5.4,7.000,7.80,8.4,9.6
Fandango_Stars_Norm,146.0,8.178082,1.080772,6.0,7.000,8.00,9.0,10.0
IMDB,146.0,6.736986,0.958736,4.0,6.300,6.90,7.4,8.6
Metacritic_Norm,146.0,5.880822,1.951739,1.3,4.350,5.90,7.5,9.4
Metacritic_User,146.0,6.519178,1.510712,2.4,5.700,6.85,7.5,9.6
RottenTomatoes_Norm,146.0,6.084932,3.016880,0.5,3.125,6.35,8.9,10.0
RottenTomatoes_User_Norm,146.0,6.387671,2.002443,2.0,5.000,6.65,8.1,9.4


Now that we have 'normed' the data the difference in mean ratings across the rating sites really stands out. (For more about this issue see [the original article](https://fivethirtyeight.com/features/fandango-movies-ratings/). 

However, what we are interested in right now is the degree to which the various ratings are associated (or how much they covary). 

In [11]:
df['Metacritic_User'].corr(df['RottenTomatoes_User'])

KeyError: 'RottenTomatoes_User'

In [ ]:
df.corr()

***
## Replacing Values

The 0 and 1 values used to the code the 'Survived' column is not easy to read or understand. The 1, 2, 3 values used to code Passenger Class are a little better but could also be improved with more descriptive values. To recode values in a column we can use the `replace()` method on a column.

In [ ]:
df['Survived'] = df['Survived'].replace(0, 'Perished')

In the first line of code above we have applied the `replace()` method to the 'Survived' column of the dataframe. Specifically, `df['Survived']` is accessing the 'Survived' column of the dataframe, and `.replace()` is calling a method on that column that takes any instance of the first argument we supply, in this case `0`, and replaces it with the second value, `'Perished'`.


We can do this again to replace `1` with `'Survived'`.

In [ ]:
df['Survived'] = df['Survived'].replace(1, 'Lived')
df.head()

We can use the same method to replace the `1`, `2`, `3` values in 'Pclass' with `'First Class'`, `'Second Class'`, and `'Third Class'`.

In [ ]:
df['Pclass'] = df['Pclass'].replace([1,2,3], ['First Class', 'Second Class', 'Third Class'])
df.head()

***
## Value Counts

That looks pretty good. Now the big question: What can this data tell us about who was likely to survive the titanic? First, let's find out how many people lived. 

In [ ]:
df['Survived'].value_counts()

What we've done here is apply the `value_counts()` method to the 'Survived' column of the dataframe.  Specifically, `df['Survived']` is accessing the 'Survived' column of the dataframe, and `.value_counts()` is calling a method on that column that counts the number of times each unique value appears in the column.


If we group dataframe rows using `.groupby()`, then `.value_counts()` will apply within each group. For example, here we group the data by the Passenger Class ('Pclass') values, then use `.value_counts()` again on the 'Survived' column of the grouped data:

In [ ]:
df_byPclass = df.groupby(by='Pclass')
df_byPclass['Survived'].value_counts()

Notice, however, that by default `value_counts()` is sorting the results by the most frequent outcome. This makes the result above a bit hard to read since the first class passengers are sorted differently than the rest (since more survived than perished). We can pass an argument to `value_counts()` to stop it from sorting this way.  

In [ ]:
df_byPclass['Survived'].value_counts(sort=False)

We can also use the `.groupby()` method to group on multiple columns by passing it a list of column names.

In [ ]:
df_byClassSex = df.groupby(by=['Pclass', 'Sex'])
df_byClassSex['Survived'].value_counts(sort=False)

We can also reverse the order of our grouping to get a slightly different output.

In [ ]:
df_bySexClass = df.groupby(by=['Sex','Pclass'])
df_bySexClass['Survived'].value_counts(sort=False)

***
## Cross Tabulation
Up to this point we have used `value_counts()` to and `groupby()` to produce basic counts in a table-like format. When we compare survival for different groups, we are taking one kind of categorical data (Survived, Perished) and seeing how it relates to another kind of categorical data (First Class, Second Class, Third Class). This type of analysis is really common in all kinds of applications. A more formal tool for looking at data this way is a ['Contingency Table' or 'Cross Tabulation'.](https://en.wikipedia.org/wiki/Contingency_table) 


In [ ]:
pd.crosstab(df['Pclass'], df['Survived'])

In the code above we have passed two columns from our dataframe into the Pandas `crosstab()` method. **Note** that this is a function in Pandas itself, not in a particular dataframe, so we are specifying `pd` (the Pandas module we imported above) on the left side of the dot notation, and we are passing dataframe columns into it as arguments.

The `crosstab()` method has some additional features that make it very useful.

First, we can add the argument `margins` that produces row or column subtotals (margins):

In [ ]:
pd.crosstab(df['Pclass'], df['Survived'], margins=True)

Second, we can add an argument `normalize` that coverts frequency counts to percentages. By setting the `normalize` argument to the string `'index'`, we specify that we want values in each row converted to percentages of that row's total.  For example, the value in the resulting table for Pclass=1 and Survived='Perished' will indicate what percentage *of first class passengers* perished:

In [ ]:
pd.crosstab(df['Pclass'], df['Survived'], margins=True, normalize='index')

Here's a similar crosstabs examining the survival of passengers with sibling or spouses aboard the ship:

In [ ]:
pd.crosstab(df['Siblings/Spouses Aboard'], df['Survived'], margins=True, normalize='index')

We can extend the cross tabs by passing a list of columns. Here we've passed in two dataframe columns for the crosstab rows and a single column for the crosstab columns.

In [ ]:
pd.crosstab([df['Pclass'], df['Sex']], df['Survived'], normalize='index')